<a href="https://colab.research.google.com/github/ishaqmarashy/Housing-Prices-ML/blob/main/Housing_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import sys
assert sys.version_info >= (3, 5)

import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import os
import urllib
import io
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Plotting
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [8]:
raw_train_url = "https://raw.githubusercontent.com/ishaqmarashy/Housing-Prices-ML/main/train.csv"
raw_test_url = "https://raw.githubusercontent.com/ishaqmarashy/Housing-Prices-ML/main/test.csv"
def load_data(url):
  # download the csv file from github
  download = requests.get(url).content
  # read the downloaded content and put into dataframe
  df = pd.read_csv(io.StringIO(download.decode("utf-8")),index_col=0,infer_datetime_format=True,dayfirst=False)
  return df

In [22]:
train_df = load_data(raw_train_url)
test_df = load_data(raw_test_url)
train_df.info() 
train_df['FireplaceQu'].value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

Gd    380
TA    313
Fa     33
Ex     24
Po     20
Name: FireplaceQu, dtype: int64

In [ ]:
# Some data is missing but we only require age from the missing values
# Drop the irrelevant data 'Name','Cabin','Ticket','Embarked'
def process(df):
  df['Sex'] = (df['Sex'] == 'female').astype(int)
  df['Age'].fillna(value=df['Age'].mean(), inplace=True)
  if df.columns.isin(['Alley','PoolQC']).any():
    df=df.drop(columns=['Alley','PoolQC'])
  # We know age and sex played an important role in the survival of the sinking titanic
  # We can use that to engineer a new feature
  return df
train_df=process(train_df)
test_df=process(test_df)
train_df.info() 
test_df.info() 